In [73]:
import pandas as pd
import sklearn as sk
import random
import numpy as np

#载入文件特征文档
features=table=pd.read_csv('item_feature.csv')


#提取训练集

data_train=pd.read_excel('train_sample_data.xlsx')
data_train["衣服ID"]=[i[:12] for i in data_train["衣服ID"]]
usr_train=data_train.groupby("客户ID")

#提取测试集
data_test=pd.read_excel('test_sample_data.xlsx')
data_test["衣服ID"]=[i[:12] for i in data_test["衣服ID"]]
usr_test=data_test.groupby("客户ID")


#训练集、测试集重新编码
def recoder(od_dataset):
    usr_item_dict={usr:[i for i in groups['衣服ID']]for usr, groups in list(od_dataset)}
    return usr_item_dict

usr_item_dict_train=recoder(usr_train)
usr_item_dict_test=recoder(usr_test)


#最热100商品提取
hot_100_items=data_train["衣服ID"].value_counts()
hot_100_items=hot_100_items[:100].index


#生成训练集
def clf_trainset_gen(history_items,hot_100_items,features):
    
    #生成反馈
    def sample_gen(history_items,features):
        frames=[]
        for i in history_items:
            frames.append(features[features['衣服ID'] == i])
        if frames:
            frames=pd.concat(frames)
        return frames
    
    #生成正反馈
    pos=sample_gen(history_items,features)
    pos['result']=1
    #生成负反馈
#     print("#########################Pos:")
#     print(pos)
    neg=sample_gen(hot_100_items,features)
    neg['result']=0
#     print("#########################Neg:")
#     print(neg)
    
    sample=pd.concat([pos,neg])
    return sample


#根据用户历史计算Rank
def rank_Compute(usr,real_item,hot_100_items):
        
        #检索用户历史
        usr_history=usr_item_dict_train[usr]
        
        #负反馈采样
        hot_100_items=[i for i in hot_100_items if i not in usr_history]
        hot_100_items=random.sample(hot_100_items, len(usr_history))
        
        
        #调用训练集函数
        clf_transet=clf_trainset_gen(usr_history,hot_100_items,features)
#         print(clf_transet)
        #生成目标变量
        
        X=clf_transet.iloc[:,2:-1]
        Y=clf_transet.iloc[:,-1]
#         print(clf_transet["result"])
#         print("XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX")
#         print(X)
#         print("YYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYYY")
#         print(Y)
        
        
        #训练模型
        #RandomForest:
        
        #Step1:Random Forest V.S Naive Bayes on Rank
        
        #Step2:Features Importance: 
        #200 VS 200 切一刀，看看是否会提高rank。如果变好，则删除一部分features
        
        #Step2:不需要特征工程
        
        #Step3:XGBoost
        
        from sklearn.naive_bayes import BernoulliNB
        CLF=BernoulliNB()
        CLF.fit(X,Y)

        #预测用户对除正负反馈外所有物品的功用
        dct={}
        for item_else in features["衣服ID"]:
            if item_else not in usr_history:
#             if item_else not in usr_history and item_else not in hot_100_items:
                XX=features[features["衣服ID"]==item_else].iloc[:,2:]
                XX
                #模型打分
                marks=CLF.predict_proba(XX)[0][0]
#                 print("#####################")
#                 print(marks)
                dct[item_else]=marks
#         print(#####################)
#         print(dct)
        lst=sorted(dct.items(),key=lambda dct:dct[1])
#         print("#####################")
#         print(lst)
        lst=[k for (k,v) in lst]
        
        #计算排名
#         print("usr",usr)
#         print('usr_history',usr_item_dict_train[usr])
#         print("real_item",real_item)
        Rank=lst.index(real_item)/len(lst)

            
            
#         print("#####################")
#         print(marks)
        print(Rank)
        return Rank
    
        
        
rank=[]
total=0
#对测试集中每一条记录，进行推荐排名
for usr_test in usr_item_dict_test:
    print("########")
    print("usr: ",usr_test)
    if usr_test in usr_item_dict_train:
        for real_item in usr_item_dict_test[usr_test]:
            total+=1
#             print(real_item)
#             print(usr_item_dict_train[usr_test])
#             print(usr_item_dict_test[usr_test])
            if real_item in usr_item_dict_train[usr_test]:
                total-=1
            else:
                rank.append(rank_Compute(usr_test,real_item,hot_100_items))
    else:
        #若无法计算则随机推荐
        rank.append(0.5)

rank_score=sum(rank)/total

print("#####################################")
print("Rank Score:", rank_score)


########
usr:  483
0.4797507788161994
0.16510903426791276
########
usr:  513
0.909375
########
usr:  781
0.006269592476489028
0.5611285266457681
########
usr:  971
0.5331230283911672
########
usr:  984
0.4968152866242038
########
usr:  1611
0.6277602523659306
0.7854889589905363
0.3217665615141956
########
usr:  1641
########
usr:  1654
0.4426751592356688
########
usr:  1746
0.18495297805642633
0.8871473354231975
########
usr:  1909
########
usr:  2187
0.5078864353312302
0.9369085173501577
0.7791798107255521
########
usr:  2344
0.09177215189873418
0.8575949367088608
0.22151898734177214
########
usr:  2700
0.0949367088607595
########
usr:  3121
########
usr:  4176
0.7183544303797469
0.43037974683544306
0.15822784810126583
0.13291139240506328
########
usr:  4303
0.334375
########
usr:  4366
0.7873015873015873
########
usr:  5902
0.79375
0.9125
########
usr:  7211
########
usr:  7339
0.8108974358974359
0.47435897435897434
0.04487179487179487
########
usr:  7722
0.2336448598130841
########
